## Register Data with Athena

In [ ]:
# import boto3
# import sagemaker

# sess = sagemaker.Session()
# bucket = sess.default_bucket()
# role = sagemaker.get_execution_role()
# region = boto3.Session().region_name


import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [ ]:
ingest_create_athena_table_csv_passed = False

In [ ]:
%store -r ingest_create_athena_db_passed

In [ ]:
try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [ ]:
print(ingest_create_athena_db_passed)

In [ ]:
if not ingest_create_athena_db_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

In [ ]:
s3_hotels_csv = "s3://projectbucketvsbr22/hotel booking/"


In [ ]:
print(ingest_create_athena_db_passed)

In [ ]:
from pyathena import connect

In [ ]:
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [ ]:
database_name = "hotels"
table_name_res = "hotel_res"

In [ ]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# Create Schema for Hotel Reservations

In [ ]:
# # SQL statement to execute

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
  `Booking_ID` char(100),
  `no_of_adults` int,
  `no_of_children` int,
  `no_of_weekend_nights` int,
  `type_of_meal_plan` char(100),
  `required_car_parking_space` int,
  `room_type_reserved` char(100),
  `lead_time` int, 
  `arrival_year` int,
  `arrival_month` int,
  `arrival_date` int, 
  `market_segment_type` char(100),
  `repeated_guest` int, 
  `no_of_previous_cancellations` int, 
  `no_of_previous_bookings_not_canceled` int,
  `avg_price_per_room` float,
  `no_of_special_request` int, 
  `booking_status` char(100)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('field.delim' = ',')
LOCATION '{}'
TBLPROPERTIES ('classification' = 'csv');""".format(
    database_name, table_name_res, s3_hotels_csv
)
print(statement)

In [ ]:
import pandas as pd

pd.read_sql(statement, conn)

In [ ]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(10)

In [ ]:
if table_name_res in df_show.values:
    ingest_create_athena_table_csv_passed = True

In [ ]:
%store ingest_create_athena_table_csv_passed

In [ ]:
statement = """SELECT * FROM {}.{}
    LIMIT 10""".format(
    database_name, table_name_res
)

print(statement)

In [ ]:
df = pd.read_sql(statement, conn)
df.head(5)

## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}